In [38]:
import pandas as pd
import numpy as np

In [39]:
df=pd.read_csv("Superstore_clean.csv")

In [41]:
df.head()

,order_id,order_date,order_year,order_month,segment,country,city,state,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit
0,CA-2016-152156,2016-08-11,2016,August,Consumer,United States,Henderson,Kentucky,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
1,CA-2016-152156,2016-08-11,2016,August,Consumer,United States,Henderson,Kentucky,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.00,219.58
2,CA-2016-138688,2016-12-06,2016,December,Corporate,United States,Los Angeles,California,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.00,2488915.00
3,US-2015-108966,2015-11-10,2015,November,Consumer,United States,Fort Lauderdale,Florida,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58,5,0.45,-383.03
4,US-2015-108966,2015-11-10,2015,November,Consumer,United States,Fort Lauderdale,Florida,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.37,2,0.20,1192184.00


In [42]:
total_sales = df['sales'].sum()

In [43]:
total_profit = df['profit'].sum()

In [44]:
df['profit_margin'] = df['profit'] / df['sales']
avg_profit_margin = df['profit_margin'].mean()

In [45]:
num_loss_orders = df[df['profit'] < 0].shape[0]
total_orders = df.shape[0]
percent_loss_orders = (num_loss_orders / total_orders) * 100

In [48]:
print(f"Total Sales: {total_sales}")
print(f"Total Profit: {total_profit}")
print(f"Average Profit Margin: {avg_profit_margin:.2%}")
print(f"% Loss Orders: {percent_loss_orders:.2f}%")

Total Sales: 44541645.86
Total Profit: 3088129378.79
Average Profit Margin: 1278026.68%
% Loss Orders: 18.73%


In [53]:
monthly_summary = df.groupby(['order_year', 'order_month']).agg(
    total_sales=('sales', 'sum'),
    total_profit=('profit', 'sum')
).reset_index()

In [55]:
for index, row in monthly_summary.iterrows():
    year = row['order_year']
    month = str(row['order_month']).zfill(2)  # додає провідний нуль, якщо потрібно
    sales = f"{row['total_sales']:.2f}"
    profit = f"{row['total_profit']:,}"    
    print(f"{year}-{month} | Sales: {sales} | Profit: {profit}")

2014-April | Sales: 633253.99 | Profit: 41,271,579.37
2014-August | Sales: 614338.58 | Profit: 55,959,931.96
2014-December | Sales: 980305.06 | Profit: 104,725,866.89
2014-February | Sales: 445242.51 | Profit: 28,082,638.59
2014-January | Sales: 711709.78 | Profit: 53,688,352.42
2014-July | Sales: 610327.73 | Profit: 62,320,813.65
2014-June | Sales: 539985.39 | Profit: 35,871,777.55
2014-March | Sales: 692147.40 | Profit: 59,673,557.3
2014-May | Sales: 403577.57 | Profit: 60,083,172.93
2014-November | Sales: 1297385.50 | Profit: 97,010,029.34
2014-October | Sales: 559058.03 | Profit: 41,907,008.35
2014-September | Sales: 1184051.19 | Profit: 74,086,897.96
2015-April | Sales: 825845.94 | Profit: 52,989,283.83
2015-August | Sales: 842743.15 | Profit: 35,837,894.35
2015-December | Sales: 1716131.21 | Profit: 86,223,226.63
2015-February | Sales: 367750.33 | Profit: 27,587,878.48
2015-January | Sales: 403667.90 | Profit: 20,278,979.71
2015-July | Sales: 712545.51 | Profit: 24,388,840.81
201

In [56]:
monthly_summary['sales_MoM_change'] = monthly_summary['total_sales'].pct_change() * 100
monthly_summary['profit_MoM_change'] = monthly_summary['total_profit'].pct_change() * 100

In [57]:
print(monthly_summary)

    order_year order_month  total_sales  total_profit  sales_MoM_change  \
0         2014       April    633253.99  4.127158e+07               NaN   
1         2014      August    614338.58  5.595993e+07         -2.987018   
2         2014    December    980305.06  1.047259e+08         59.570812   
3         2014    February    445242.51  2.808264e+07        -54.581229   
4         2014     January    711709.78  5.368835e+07         59.847670   
5         2014        July    610327.73  6.232081e+07        -14.244858   
6         2014        June    539985.39  3.587178e+07        -11.525339   
7         2014       March    692147.40  5.967356e+07         28.178912   
8         2014         May    403577.57  6.008317e+07        -41.691962   
9         2014    November   1297385.50  9.701003e+07        221.471161   
10        2014     October    559058.03  4.190701e+07        -56.908873   
11        2014   September   1184051.19  7.408690e+07        111.793969   
12        2015       Apri

In [58]:
category_summary = (
    df.groupby(['category', 'subcategory'])
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_id', 'nunique')
      )
      .reset_index()
)

category_summary

,category,subcategory,total_sales,total_profit,avg_profit_margin,orders_count
0,Furniture,Bookcases,114880.05,1.746738e+07,530.633371,224
1,Furniture,Chairs,328449.13,4.631399e+07,699.820589,576
2,Furniture,Furnishings,3959851.12,3.811910e+08,14143.596941,877
3,Furniture,Tables,206965.68,1.701850e+07,196.747682,307
4,Office Supplies,Appliances,1074061.44,9.792410e+07,5611.754397,451
5,Office Supplies,Art,9175250.65,4.297092e+08,14474.037454,731
6,Office Supplies,Binders,6622988.60,3.639537e+08,14101.880896,1316
7,Office Supplies,Envelopes,1313071.59,1.074644e+08,20225.336014,249
8,Office Supplies,Fasteners,3413481.87,1.205192e+08,25112.501335,215
9,Office Supplies,Labels,5029283.33,2.689257e+08,44719.898941,346


In [59]:
region_summary = (
    df.groupby('region')
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_id', 'nunique')
      )
      .reset_index()
)

region_summary

,region,total_sales,total_profit,avg_profit_margin,orders_count
0,Central,8519887.69,6.180132e+08,13166.344424,1174
1,East,12610566.06,9.169909e+08,13806.219953,1401
2,South,6073679.99,4.989899e+08,12963.926711,822
3,West,17337512.12,1.054135e+09,11495.924004,1611


In [60]:
segment_summary = (
    df.groupby('segment')
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_id', 'nunique')
      )
      .reset_index()
)

segment_summary

,segment,total_sales,total_profit,avg_profit_margin,orders_count
0,Consumer,22975071.16,1.625222e+09,12640.956573,2585
1,Corporate,13422765.25,9.381960e+08,13227.708219,1514
2,Home Office,8143809.45,5.247117e+08,12427.369053,909


In [62]:
category_summary.sort_values('total_profit', ascending=False)

,category,subcategory,total_sales,total_profit,avg_profit_margin,orders_count
10,Office Supplies,Paper,7945558.41,5.846059e+08,25583.606732,1191
5,Office Supplies,Art,9175250.65,4.297092e+08,14474.037454,731
2,Furniture,Furnishings,3959851.12,3.811910e+08,14143.596941,877
6,Office Supplies,Binders,6622988.60,3.639537e+08,14101.880896,1316
11,Office Supplies,Storage,966421.47,2.956687e+08,7443.441764,777
9,Office Supplies,Labels,5029283.33,2.689257e+08,44719.898941,346
16,Technology,Phones,983176.09,1.547885e+08,4060.248809,814
13,Technology,Accessories,1497195.00,1.394563e+08,5433.500324,713
8,Office Supplies,Fasteners,3413481.87,1.205192e+08,25112.501335,215
7,Office Supplies,Envelopes,1313071.59,1.074644e+08,20225.336014,249


In [63]:
region_summary.sort_values('total_profit', ascending=False)

,region,total_sales,total_profit,avg_profit_margin,orders_count
3,West,17337512.12,1.054135e+09,11495.924004,1611
1,East,12610566.06,9.169909e+08,13806.219953,1401
0,Central,8519887.69,6.180132e+08,13166.344424,1174
2,South,6073679.99,4.989899e+08,12963.926711,822


In [64]:
segment_summary.sort_values('total_profit', ascending=False)

,segment,total_sales,total_profit,avg_profit_margin,orders_count
0,Consumer,22975071.16,1.625222e+09,12640.956573,2585
1,Corporate,13422765.25,9.381960e+08,13227.708219,1514
2,Home Office,8143809.45,5.247117e+08,12427.369053,909
